In [7]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from bs4 import BeautifulSoup 
from tqdm import tqdm
import requests
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [8]:
camping_num   = []  # 캠핑장 고유넘버

camping_type  = []  # 캠핑장 유형 (ex.캠핑,야영장 / 온천,스파 / 카페 등등..)
camping_name  = []  # 캠핑장 이름
camping_addr  = []  # 캠핑장 주소
camping_score = []  # 캠핑장 평균별점
camping_phone = []  # 캠핑장 전화번호
camping_site  = []  # 캠핑장 사이트(site) 
camping_info1 = []  # 캠핑장 부가설명1
camping_info2 = []  # 캠핑장 부가설명2


## 고유넘버 크롤링

In [9]:
def num_crawl(keyword):
    # 검색할 단어 입력
    #keyword = '경기도 캠핑, 야영장'
    url = f'https://map.naver.com/v5/search/{keyword}/place/'

    driver = webdriver.Chrome('/Users/ds/chromedriver 2')
    driver.get(url)
    time.sleep(2)

    # iframe 변경
    driver.switch_to.frame('searchIframe')

    for num in tqdm(range(1,7)): # 반복을 6번
        # END 키를 눌러서 페이지 제일 하단으로 이동
        driver.find_element_by_xpath('//*[@id="_pcmap_list_scroll_container"]').click()
        if num == 1:
            pass
        else:
            driver.find_element_by_xpath(f'//*[@id="app-root"]/div/div[2]/div[2]/a[7]').click()

        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        time.sleep(2)
        camping_list = soup.findAll('li', {'class':'_3t81n _1l5Ut _19sVg'})

        for i in tqdm(range(len(camping_list))):
            driver.find_element_by_xpath(f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{str(i+1)}]/div[1]/a').send_keys(Keys.ENTER)
            time.sleep(2)

            current_url = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지

            unique_code = re.findall(r"place/(\d+)", current_url)
            camping_num.append(unique_code[0])
            time.sleep(2)
        

        

    driver.quit()# 검색할 단어 입력

In [11]:
num_crawl('제주도 맛집')

/var/folders/sy/lzv1b4r92bqb6hy15tk2qy5h0000gn/T/ipykernel_53724/3782680100.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/ds/chromedriver 2')


WebDriverException: Message: 'chromedriver 2' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [26]:
camping_list

NameError: name 'camping_list' is not defined

## 캠핑장정보 크롤링

In [16]:
def info_crawl():
    driver = webdriver.Chrome('../driver/chromedriver')
    for i in tqdm(camping_num):
        new_url = f'https://pcmap.place.naver.com/accommodation/{i}/home?'
        driver.get(new_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        time.sleep(1)
        name = soup.find('span',{'class':'_3XamX'})
        style = soup.find('span',{'class':'_3ocDE'})
        addr = soup.find('span',{'class':'_2yqUQ'})
        score = soup.find('span',{'class':'_1Y6hi _1A8_M'})
        try :
            camping_type.append(style.text)     ## 캠핑장 유형
            camping_name.append(name.text)      ## 캠핑장 이름
            camping_addr.append(addr.text)      ## 캠핑장 주소
            camping_score.append(score.em.text) ## 캠핑장 평균별점
        except:
            camping_score.append('none')
        ## 전화번호
        try:     
            phone_num = soup.find('span',{'class':'_3ZA0S'})
            camping_phone.append(phone_num.text)        
        except:
            camping_phone.append('none')  
        ## 캠핑장 사이트(site) 
        try:
            site = soup.find('div',{'class':'_2SBb1'})
            camping_site.append(site.text)        
        except:
            camping_site.append('none')
        ## 부가설명
        info = soup.findAll('li',{'class':'_1M_Iz'})
        s = -1 # 부가설명1
        t = -1 # 부가설명2
        for i in info:
            try:
                a = i.find('span',{'class':'place_blind'}).text
            except:
                a = ""
            print(a)
            ## 부가설명1
            if a == '편의':
                s = 0
                text1 = i.find('div',{'class':'_1h3B_'}).text
            ## 부가설명2
            if a == '설명':
                t = 0
                text2 = i.find('div',{'class':'_1h3B_'}).text
        if s == 0:
            camping_info1.append(text1)
        else:
            camping_info1.append('none')
        if t == 0:
            camping_info2.append(text2)
        else:
            camping_info2.append('none')
    driver.quit()
    
    camping_dict = {'Num':camping_num, 'Type':camping_type, 'Name':camping_name, 'Address':camping_addr, 'Score':camping_score, 'Phone':camping_phone, 'URL':camping_site, 'info1':camping_info1, 'info2':camping_info2}
    camp_info_df = pd.DataFrame(camping_dict)
    camp_info = pd.DataFrame()
    for i in range(len(camp_info_df)):
        if camp_info_df['Type'][i] == '캠핑,야영장':
            camp_info = camp_info.append(camp_info_df.loc[i])
    camp_info = camp_info[['Num','Type', 'Name', 'Address', 'Phone', 'Score','URL','info1','info2']]
    camp_info.index = range(0,len(camp_info))
    camp_info['Num'] = camp_info['Num'].astype('int')


In [18]:
info_crawl()

/var/folders/zz/lwjzp_r130b4y7w9qcwwkfhr0000gn/T/ipykernel_83785/855970963.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../driver/chromedriver')


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [19]:
camping_dict = {'Num':camping_num, 'Type':camping_type, 'Name':camping_name, 'Address':camping_addr, 'Score':camping_score, 'Phone':camping_phone, 'URL':camping_site, 'info1':camping_info1, 'info2':camping_info2}
camp_info_df = pd.DataFrame(camping_dict)
gyeonggi_camp_info = pd.DataFrame()
for i in range(len(camp_info_df)):
    if camp_info_df['Type'][i] == '캠핑,야영장':
        gyeonggi_camp_info = gyeonggi_camp_info.append(camp_info_df.loc[i])
gyeonggi_camp_info = gyeonggi_camp_info[['Num','Type', 'Name', 'Address', 'Phone', 'Score','URL','info1','info2']]
gyeonggi_camp_info.index = range(0,len(gyeonggi_camp_info))
gyeonggi_camp_info['Num'] = gyeonggi_camp_info['Num'].astype('int')



ValueError: All arrays must be of the same length

In [20]:
gyeonggi_camp_info

NameError: name 'gyeonggi_camp_info' is not defined

# 캠핑장 리뷰 크롤링

In [21]:
camping_num_list  = []
camping_addr_list = []
camping_name_list = []
user_id_list      = []
camping_type_list = []
rating_list       = []
avg_rating_list   = []
date_list         = []
review_list       = []
count_list        = []
error             = [] # 에러 내용

In [ ]:
def review_crawl():
    driver = webdriver.Chrome('../driver/chromedriver.exe')

    for i in range(6,8):

        review_url = 'https://pcmap.place.naver.com/accommodation/{}/review/visitor?'.format(gyeonggi_camp_info['Num'][i])
    #     print(review_url)
        try:
            print('======================================================')
            print(str(i+1)+'번째 캠핑장')

            # 캠핑장 리뷰 개별 url 접속
            driver.get(review_url)
            time.sleep(2) 

            while True:
                try:
                    # END 키를 눌러서 페이지 제일 하단으로 이동
                    driver.find_element_by_tag_name('body').send_keys(Keys.END)
                    time.sleep(3)
                    # 더보기 버튼 클릭
                    driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[4]/div[2]/a').click()
                    time.sleep(3)
                    # END 키를 눌러서 페이지 제일 하단으로 이동
                    driver.find_element_by_tag_name('body').send_keys(Keys.END)
                    time.sleep(1)

                except NoSuchElementException:
                    print('-더보기 버튼 모두 클릭 완료-')
                    break

            # 파싱
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
            time.sleep(1)

            # 캠핑장 구분
            camping_site_name = gyeonggi_camp_info['Name'][i]
            print('사업장명 : '+ camping_site_name)

            try:
                all_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
                review_num = len(all_review) # 특정 캠핑의 리뷰 총 개수
                print('리뷰 총 개수 : '+ str(review_num))

                print('----------------------------------------------')

                for review in range(review_num):
                    ## 캠핑장 고유 넘버 
                    camping_num_list.append(gyeonggi_camp_info["Num"][i])

                    ## 캠핑장주소
                    camping_addr_list.append(gyeonggi_camp_info["Address"][i])                         

                    ## 캠핑장 이름
                    camping_name_list.append(camping_site_name)
                    print(camping_site_name)

                    ## user ID
                    user = soup.find_all('div',{'class':'hbo4A'})
                    print('{}번째 user:'.format(review+1),user[review].text)
                    user_id_list.append(user[review].text)

                    ## rating(사용자가 이 캠핑장에 준 별점)
                    rating = all_review[review].find('span', attrs = {'class':'_2tObC'}).text
                    print('rating = '+rating)
                    rating_list.append(rating)

                    ## 사용자의 평균 별점
                    avg_rating = soup.find_all('div',{'class':'ql4ZC'})
                    print('avg_rating ='+avg_rating[review].text)
                    avg_rating_list.append(avg_rating[review].text)

                    # 주의!!! 사진 리뷰 유무에 따라 날짜 파싱코드 다름
                    # ('span', attrs = {'class':'_3WqoL'})
                    # 사진 없는 경우 : 총 6개 중 4번째
                    # 사진 있는 경우 : 총 5개 중 3번째

                    ## date
                    # 사진 리뷰 없음
                    if len(all_review[review].find_all('span', attrs = {'class':'_3WqoL'})) == 5:
                        date = all_review[review].find_all('span', attrs = {'class':'_3WqoL'})[2].text
                    elif len(all_review[review].find_all('span', attrs = {'class':'_3WqoL'})) == 6:
                        date = all_review[review].find_all('span', attrs = {'class':'_3WqoL'})[3].text
                    else:
                        date = ""
                    print('date = '+date)
                    date_list.append(date)

                    ## count 방문횟수
                    if len(all_review[review].find_all('span', attrs = {'class':'_3WqoL'})) == 5:
                        count = all_review[review].find_all('span', attrs = {'class':'_3WqoL'})[3].text
                    elif len(all_review[review].find_all('span', attrs = {'class':'_3WqoL'})) == 6:
                        count = all_review[review].find_all('span', attrs = {'class':'_3WqoL'})[4].text
                    else:
                        count = ""
                    print('방문횟수 = '+count)
                    count_list.append(count)                

                    ## review 내용
                    try: 
                        review_content = all_review[review].find('span', attrs = {'class':'WoYOw'}).text
                    except: # 리뷰가 없다면
                        review_content = ""
                    print('리뷰 내용 : '+review_content)
                    review_list.append(review_content)                    

                    print('\n\n')

            except Exception as total_review_err:
                none_review = "네이버 리뷰 0개"
                print(none_review)   


        except Exception as err:
            print(err)
            error_list = '{}행 에러 : {}'.format(i,err)
            error.append(error_list)
            rating_list.append(' ')
            avg_rating_list.append(' ')
            review_list.append(' ')

    driver.quit()

In [ ]:
review_crawl()

In [ ]:
review_data = {'Num':camping_num_list,'Address':camping_addr_list,'Date':date_list, 'Name':camping_name_list, 'User_ID':user_id_list, 'Rating':rating_list, 
               'Avg_rating':avg_rating_list, 'Review':review_list}
review_df = pd.DataFrame(review_data)
review_df

review = []
avg_ratings = []
for i in review_df['Avg_rating'].tolist():
    a = re.findall('\d+[.]?\d*', i)
    try:
        review.append(a[0])
        avg_ratings.append(a[-1])    
    except:
        review.append('none')
        avg_ratings.append('none')  

review_df['Total_Review']=review
review_df['User_Avg_Ratings']=avg_ratings

review_final = review_df[['Num','Date','Name','Address','User_ID','Total_Review','User_Avg_Ratings','Rating','Review']]
review_final

In [ ]:
review_final.to_csv('review_final.csv')